<a href="https://colab.research.google.com/github/junxnone/examples/blob/master/sigopt/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install sigopt

In [ ]:
!sigopt config
import sigopt
%load_ext sigopt

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn import metrics
import sigopt

In [13]:

def train():
    mnist = keras.datasets.mnist
    (train_images, train_labels), (val_images, val_labels) = mnist.load_data()

    train_images = train_images / 255.0
    val_images = val_images / 255.0

    # Define Model
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Reshape(target_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10)
    ])
    sel_optimizer = sigopt.params.optimizer
    sel_batchsize = sigopt.params.batch_size
    sel_init_lr = sigopt.params.lr
    print(f' select optimizer/batchsize/learing_rate {sel_optimizer} / {sel_batchsize} /{sel_init_lr}')

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=sel_init_lr,
      decay_steps=50,
      decay_rate=0.9)
    if sel_optimizer == 'SGD':
      optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
    elif sel_optimizer == 'Adam':
      optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(train_images, train_labels, batch_size=sel_batchsize, epochs=5, validation_data=(val_images, val_labels))


    # Evalution
    preds = model.predict(val_images)
    preds_sort = np.argsort(-preds, axis=1)

    preds_labels = []
    for i in range(preds_sort.shape[0]):
        preds_labels.append(preds_sort[i][0])

    acc = metrics.accuracy_score(val_labels, preds_labels)
    print(f'Accuracy is {acc}')

    recall = metrics.recall_score(val_labels, preds_labels, average='macro')
    print(f'Recall is {recall}')

    precision = metrics.precision_score(val_labels, preds_labels, average='macro')
    print(f'Precision is {precision}')

    top3_acc = metrics.top_k_accuracy_score(val_labels, preds, k = 3)

    print(f'top 3 acc is {top3_acc}')
 
    return acc, top3_acc, recall, precision

In [ ]:
%%experiment
{
    'name': 'mnist colab',
    'metrics': [
        {
            'name': 'accuracy',
            'strategy': 'optimize',
            'objective': 'maximize',
        },
        {
            'name': 'recall',
            'strategy': 'optimize',
            'objective': 'maximize',
        }
    ],
    'parameters': [
        {
            'name': 'batch_size',
            'type': 'int',
            'grid': [32, 64, 128, 256, 512]
        },
        {
            'name': 'lr',
            'type': 'double',
            'bounds': {'min': 0.0001, 'max': 0.8}
        },
        {
            'name': 'optimizer',
            'type': 'categorical',
            'categorical_values': ["Adam", "SGD"]
        }
    ],
    'budget': 20
}

In [ ]:
%%optimize mnist_colab

try:

  # train
  sigopt.log_dataset("Mnist")
  sigopt.log_model(type="CNN")
  sigopt.log_metadata(key="Dataset Source", value="tensorflow.keras.datasets")
  val_acc, top3_acc, recall, precision = train()
  sigopt.log_metric(name='accuracy', value=val_acc)
  sigopt.log_metric(name='recall', value=recall)
except Exception as e:
  LOG.exception(e)
  raise